## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-26-04-02-30 +0000,nyt,"George Cabot Lodge, Last of His Family to Batt...",https://www.nytimes.com/2026/01/25/obituaries/...
1,2026-01-26-04-01-00 +0000,bbc,Heatwave warnings across Australia as millions...,https://www.bbc.com/news/articles/cz6y7nxx5jvo...
2,2026-01-26-04-00-00 +0000,wsj,Iran Is Selling More Oil but Making Less Money,https://www.wsj.com/world/middle-east/iran-is-...
3,2026-01-26-04-00-00 +0000,wsj,"Here in One Kyiv Apartment Building, They Are ...",https://www.wsj.com/world/here-in-one-kyiv-apa...
4,2026-01-26-03-44-43 +0000,wapo,Minneapolis live updates: Border Patrol sidest...,https://www.washingtonpost.com/nation/2026/01/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2446/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
37,shooting,30
161,trump,29
29,minneapolis,26
48,alex,18
49,pretti,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
23,2026-01-26-01-57-16 +0000,nypost,Trump ‘reviewing everything’ about fatal shoot...,https://nypost.com/2026/01/25/us-news/trump-re...,154
71,2026-01-25-22-29-51 +0000,nypost,LA’s top fed prosecutor Bill Essayli warns aga...,https://nypost.com/2026/01/25/us-news/las-bill...,137
177,2026-01-25-10-12-37 +0000,nypost,Senate Dems could force gov’t shutdown over $6...,https://nypost.com/2026/01/25/us-news/senate-d...,111
141,2026-01-25-15-31-54 +0000,bbc,Video shows moments around fatal shooting of A...,https://www.bbc.com/news/videos/cp372pqq2rlo?a...,111
204,2026-01-25-04-19-46 +0000,bbc,What we know about fatal shooting of Alex Pret...,https://www.bbc.com/news/articles/c20zjyxep99o...,110


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
23,154,2026-01-26-01-57-16 +0000,nypost,Trump ‘reviewing everything’ about fatal shoot...,https://nypost.com/2026/01/25/us-news/trump-re...
89,45,2026-01-25-21-25-09 +0000,startribune,"Department of Justice renews push for voter, w...",https://www.startribune.com/department-of-just...
45,44,2026-01-26-00-14-51 +0000,nypost,"Winter Storm Fern rages across US, canceling n...",https://nypost.com/2026/01/25/us-news/winter-s...
63,36,2026-01-25-22-48-00 +0000,wsj,Senate Democrats angered by the deadly shootin...,https://www.wsj.com/politics/policy/senate-dem...
197,31,2026-01-25-06-02-10 +0000,nypost,Florida Rep. Maxwell Frost says he was punched...,https://nypost.com/2026/01/25/us-news/florida-...
138,30,2026-01-25-15-41-03 +0000,cbc,"No plans for China free trade deal, Carney say...",https://www.cbc.ca/news/politics/carney-canada...
71,29,2026-01-25-22-29-51 +0000,nypost,LA’s top fed prosecutor Bill Essayli warns aga...,https://nypost.com/2026/01/25/us-news/las-bill...
168,27,2026-01-25-11-00-39 +0000,nypost,NYC homeless living high life under Mayor Mamd...,https://nypost.com/2026/01/25/us-news/nyc-home...
125,26,2026-01-25-17-00-00 +0000,wsj,U.S. Speaker of the House Mike Johnson has hel...,https://www.wsj.com/politics/republicans-house...
60,25,2026-01-25-23-09-32 +0000,wapo,ICU nurse fatally shot by Border Patrol in Min...,https://www.washingtonpost.com/nation/2026/01/...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
